In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [28]:
# Display all available columns in the dataframes
pd.set_option("display.max_columns", None)

### load TBS dataset 

In [ ]:

tbaustoff_df = pd.read_csv('data/tBaustoff/tBaustoff_with_OBD_mapping.csv', low_memory=False)

print(tbaustoff_df.shape)
tbaustoff_df.head(1)

(815, 11)


,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor
0,3b1f0c75-07a4-4182-b310-5529fc5b54a6,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.0


### load OBD datasets

In [7]:
# Load all three OBD files (adjust paths as needed)

obd_2020_df = pd.read_csv('data/OBD/OBD_2020_II.csv',
                            sep=';', encoding='ISO-8859-1', low_memory=False)
obd_2023_df = pd.read_csv('data/OBD/OBD_2023_I.csv',
                            sep=';', encoding='ISO-8859-1', low_memory=False)
obd_2024_df = pd.read_csv('data/OBD/OBD_2024_I.csv',
                            sep=';', encoding='ISO-8859-1', low_memory=False)

# Add column with DB version
obd_2020_df['OBD_Version'] = 'OBD_2020_II'
obd_2023_df['OBD_Version'] = 'OBD_2023_I'
obd_2024_df['OBD_Version'] = 'OBD_2024_I'

# Combine all OBD datasets into one DataFrame
OBD_all_years_df = pd.concat([obd_2020_df, obd_2023_df, obd_2024_df], ignore_index=True)
print(OBD_all_years_df.shape)
OBD_all_years_df.head(2)

(40411, 83)


,UUID,Version,Name (de),Name (en),Kategorie (original),Kategorie (en),Konformität,Laenderkennung,Typ,Referenzjahr,...,SOP (A2),ODP (A2),POCP (A2),ADPF (A2),ADPE (A2),WDP (A2),Unnamed: 79,OBD_Version,Stueckgewicht (kg),Unnamed: 80
0,6b2b9708-14d8-4f3a-8a75-0b7481c5e0f8,00.03.000,Innenverzinnte Kupfer-Hausinstallationsrohre,NaN,'Metalle' / 'Kupfer' / 'Kupferrohre','Metals' / 'Copper' / 'Copper pipes','DIN EN 15804' / 'ISO 14025',DE,average dataset,2015.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OBD_2020_II,NaN,NaN
1,6b2b9708-14d8-4f3a-8a75-0b7481c5e0f8,00.03.000,Innenverzinnte Kupfer-Hausinstallationsrohre,NaN,'Metalle' / 'Kupfer' / 'Kupferrohre','Metals' / 'Copper' / 'Copper pipes','DIN EN 15804' / 'ISO 14025',DE,average dataset,2015.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OBD_2020_II,NaN,NaN


### TBS with UUIDs details

In [23]:
# TBS unique UUID (assumed as mapped to OBDs) = 728
TBS_with_UUIDs_df = tbaustoff_df[tbaustoff_df['oekobaudatProcessUuid'].notna()]
print(TBS_with_UUIDs_df.shape)

TBS_unique_UUID=set(TBS_with_UUIDs_df['oekobaudatProcessUuid'].unique())
len(TBS_unique_UUID)

(728, 11)


728

### Merge TBS with UUIDs details, with OBDs corresponding details

#### only keep unique rows by UUIDs

In [11]:
OBD_all_years_df_LessFeatures=OBD_all_years_df[['UUID', 'Name (de)', 'Name (en)', 'Kategorie (original)', 'Kategorie (en)', 'Typ', 'OBD_Version','Referenzjahr', 'Gueltig bis','UUID des Vorgängers',
                                        'Bezugsgroesse', 'Bezugseinheit', 'Referenzfluss-UUID','Referenzfluss-Name', 'Schuettdichte (kg/m3)','Flaechengewicht (kg/m2)',
                                        'Rohdichte (kg/m3)', 'Schichtdicke (m)']]
OBD_all_years_df_LessFeatures.shape

(40411, 18)

In [25]:
print(OBD_all_years_df_LessFeatures.duplicated().sum())

OBD_all_years_df_LessFeatures_2=OBD_all_years_df_LessFeatures.drop_duplicates()
OBD_all_years_df_LessFeatures_2.shape

34626


(5785, 18)

#### merge TBS with OBDs

In [ ]:
# map OBD into TBS
combined_TBS_OBD = TBS_with_UUIDs_df.merge(OBD_all_years_df_LessFeatures_2, left_on=['oekobaudatProcessUuid','oekobaudatDatastockName'], right_on=['UUID','OBD_Version'], how='left')
print(combined_TBS_OBD.shape)
combined_TBS_OBD.head(2)

(728, 29)


,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,...,Gueltig bis,UUID des Vorgängers,Bezugsgroesse,Bezugseinheit,Referenzfluss-UUID,Referenzfluss-Name,Schuettdichte (kg/m3),Flaechengewicht (kg/m2),Rohdichte (kg/m3),Schichtdicke (m)
0,3b1f0c75-07a4-4182-b310-5529fc5b54a6,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,...,2024.0,3b1f0c75-07a4-4182-b310-5529fc5b54a6,1,m3,ea4a7831-456f-4b70-9c5a-a21cd816acb9,Acetylated Wood (5% water content) m3,NaN,NaN,510.0,NaN
1,0dec19c7-53db-4cb9-bbba-4d44d3da62a9,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,...,2026.0,3b1f0c75-07a4-4182-b310-5529fc5b54a6,1,m3,ea4a7831-456f-4b70-9c5a-a21cd816acb9,Acetylated Wood (5% water content) m3,NaN,NaN,510,NaN


##### overview of the combined TBS and OBDs dataset

In [30]:
def describe_columns_unique_nan(df):
    cat_cols = df.columns
    result = pd.DataFrame({
        'column': cat_cols,
        'count': [df[col].count() for col in cat_cols],
        'nunique': [df[col].nunique() for col in cat_cols],
        'null_count': [df[col].isnull().sum() for col in cat_cols],
        'null_percent': [round(df[col].isnull().sum() / len(df) * 100, 2) for col in cat_cols],
        'sample_values': [', '.join(df[col].dropna().sample(min(3, df[col].nunique())).astype(str).tolist()) for col in cat_cols]
    })
    
    return result

all_columns_unique_nan_summary = describe_columns_unique_nan(combined_TBS_OBD)
all_columns_unique_nan_summary.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
column,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor,UUID,Name (de),Name (en),Kategorie (original),Kategorie (en),Typ,OBD_Version,Referenzjahr,Gueltig bis,UUID des Vorgängers,Bezugsgroesse,Bezugseinheit,Referenzfluss-UUID,Referenzfluss-Name,Schuettdichte (kg/m3),Flaechengewicht (kg/m2),Rohdichte (kg/m3),Schichtdicke (m)
count,728,728,728,728,728,728,728,728,728,728,728,698,698,698,698,698,698,698,698,698,490,698,698,698,698,3,196,366,0
nunique,728,3,3,260,1,260,107,107,10,10,5,698,336,333,129,131,1,3,3,3,262,9,5,249,249,2,107,150,0
null_count,0,0,0,0,0,0,0,0,0,0,0,30,30,30,30,30,30,30,30,30,238,30,30,30,30,725,532,362,728
null_percent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.12,4.12,4.12,4.12,4.12,4.12,4.12,4.12,4.12,32.69,4.12,4.12,4.12,4.12,99.59,73.08,49.73,100.0
sample_values,"ba58d8b9-945b-4428-b4e0-aecda8b3df18, aaf4baca...","22885a6e-1765-4ade-a35e-ae668bd07256, 448d1096...","OBD_2023_I, OBD_2023_I, OBD_2024_I","294, 168, 12",2024-Q4,"Kalkfarbe, Kalk-Gips-Innenputz auf Gipsbaustof...","11, 53, 13","Holzwerkstoff - nicht beschichtet, Bims / Bläh...","EB, Dep+, EV-","Dep-, RC-, EV-","0.0, 0.25, 0.75","c7e171fb-d5eb-4867-9fb2-961dec81d6aa, 531ab34f...","Stahl warmgewalzte Bleche (2-20mm), Hüttenstei...","Aluminium sheet, Mineral fibre ceiling board (...",'Metalle' / 'Aluminium' / 'Gussteile aus Alumi...,"'Plastics' / 'Sealing materials' / 'Bitumen', ...",generic dataset,"OBD_2023_I, OBD_2024_I, OBD_2020_II","2022.0, 2023.0, 2023.0","2026.0, 2026.0, 2024.0","2632f4e8-7671-4d87-a8bc-9defdb54d117, d171cb17...","1, 1, 1","m3, kg, m","16da6ae2-d368-4bf8-9111-41b8297e33cd, 80e94985...","PVC-Dachbahn (m2), Polyurethan (PU) Hartschaum...","500.0, 500.0","6.2, 104.0, 1.2","2306.0, 26.25, 2360",


from the above overview, we can see that 30 unique UUIDs in TBS is not found in any of the 3 OBD datasets



#### adjust combined dataframe of TBS with OBDs

###### UUID in TBS not found in any of the 3 OBDs

In [ ]:
# find the UUIDs that are not mapped to OBDs
emptyUUID=combined_TBS_OBD[combined_TBS_OBD['UUID'].isna()]
print(emptyUUID.shape)
emptyUUID.head(3)

(30, 29)


,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor,UUID,Name (de),Name (en),Kategorie (original),Kategorie (en),Typ,OBD_Version,Referenzjahr,Gueltig bis,UUID des Vorgängers,Bezugsgroesse,Bezugseinheit,Referenzfluss-UUID,Referenzfluss-Name,Schuettdichte (kg/m3),Flaechengewicht (kg/m2),Rohdichte (kg/m3),Schichtdicke (m)
11,b8a9dd13-100b-40bf-b982-06937c4c5f65,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,101,2024-Q4,Glasfaser-Armierung,3,"Bitumenkaltkleber, Glasvlies, Glasfaserarmierung",EB,EB,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,b6e523cc-d329-49fb-b138-af531552862f,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,24,2024-Q4,Polyisobutylen (PIB) Kunststoffdachbahn,16,KSTBahn,EV-,EV-,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,7b26b760-0cc0-48b1-8c61-9357cd2b7ba3,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,67,2024-Q4,Expandierte Perlit-Dämmplatten für den Innenbe...,18,Mineralplatte,Dep+,SV,0.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# remove empty UUIDs from combined dataframe TBS OBD
combined_TBS_OBD_2=combined_TBS_OBD[combined_TBS_OBD['UUID'].notna()]
print(combined_TBS_OBD_2.duplicated().sum())
print(combined_TBS_OBD_2.shape)
combined_TBS_OBD_2.head(3)

0
(698, 29)


,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor,UUID,Name (de),Name (en),Kategorie (original),Kategorie (en),Typ,OBD_Version,Referenzjahr,Gueltig bis,UUID des Vorgängers,Bezugsgroesse,Bezugseinheit,Referenzfluss-UUID,Referenzfluss-Name,Schuettdichte (kg/m3),Flaechengewicht (kg/m2),Rohdichte (kg/m3),Schichtdicke (m)
0,3b1f0c75-07a4-4182-b310-5529fc5b54a6,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.00,3b1f0c75-07a4-4182-b310-5529fc5b54a6,"Acetyliertes Holz (1 m³, 510 kg/m³)","Acetylated wood (1 m³, 510 kg/m³)",'Holz' / 'Modifiziertes Holz' / 'Acetyliertes ...,'Wood' / 'End-of-life processes' / 'EoL partic...,generic dataset,OBD_2023_I,2022.0,2024.0,3b1f0c75-07a4-4182-b310-5529fc5b54a6,1,m3,ea4a7831-456f-4b70-9c5a-a21cd816acb9,Acetylated Wood (5% water content) m3,NaN,NaN,510.0,NaN
1,0dec19c7-53db-4cb9-bbba-4d44d3da62a9,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.00,0dec19c7-53db-4cb9-bbba-4d44d3da62a9,"Acetyliertes Holz (1 m3, 510 kg/m3)","Acetylated wood (1 m3, 510 kg/m3)",'Holz' / 'Modifiziertes Holz' / 'Acetyliertes ...,'Wood' / 'Modified wood' / 'Acetylated wood',generic dataset,OBD_2024_I,2023.0,2026.0,3b1f0c75-07a4-4182-b310-5529fc5b54a6,1,m3,ea4a7831-456f-4b70-9c5a-a21cd816acb9,Acetylated Wood (5% water content) m3,NaN,NaN,510,NaN
2,bd6d6d89-b76d-4002-a217-afffbb8aa308,448d1096-2017-4901-a560-f652a83c737e,OBD_2020_II,2,2024-Q4,Aluminium Profil,2,Alu unbeschichtet od. eloxiert (Sz. Knetleg zu...,RC+,CL+,0.75,bd6d6d89-b76d-4002-a217-afffbb8aa308,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles',generic dataset,OBD_2020_II,2018.0,2022.0,NaN,1,kg,fa4d0c5b-122a-44db-b6c3-ce03a0c6f794,Aluminium-Extrusionsprofil,NaN,NaN,2700.0,NaN


#### overview of the current UUIDs in TBS mapped with the 3 OBDs

In [36]:
all_columns_unique_nan_summary = describe_columns_unique_nan(combined_TBS_OBD_2)
all_columns_unique_nan_summary.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
column,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor,UUID,Name (de),Name (en),Kategorie (original),Kategorie (en),Typ,OBD_Version,Referenzjahr,Gueltig bis,UUID des Vorgängers,Bezugsgroesse,Bezugseinheit,Referenzfluss-UUID,Referenzfluss-Name,Schuettdichte (kg/m3),Flaechengewicht (kg/m2),Rohdichte (kg/m3),Schichtdicke (m)
count,698,698,698,698,698,698,698,698,698,698,698,698,698,698,698,698,698,698,698,698,490,698,698,698,698,3,196,366,0
nunique,698,3,3,260,1,260,107,107,10,10,5,698,336,333,129,131,1,3,3,3,262,9,5,249,249,2,107,150,0
null_count,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,208,0,0,0,0,695,502,332,698
null_percent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.8,0.0,0.0,0.0,0.0,99.57,71.92,47.56,100.0
sample_values,"f9f742db-d695-456a-9793-0636eab7df05, d5df3d3b...","22885a6e-1765-4ade-a35e-ae668bd07256, 22885a6e...","OBD_2020_II, OBD_2020_II, OBD_2020_II","85, 314, 108",2024-Q4,"Edelstahlblech, Lehmbauplatte mit CL-Nachweis,...","10, 110, 16","DS Pflanzenfaser mit Zusatzstoffen (FSM, Binde...","Dep+, RC+, EV-","CL+, RC+, RC-","0.5, 1.0, 1.0","422b2446-8a3f-457d-b47b-4728b2869d86, 286b0072...","PE-Noppenfolie zur Abdichtung, Unterspannbahn ...","Polystyrene foam filling (without binder), Ure...",'Mineralische Baustoffe' / 'Zuschläge' / 'Sand...,'Mineral building products' / 'Asphalt' / 'Bas...,generic dataset,"OBD_2023_I, OBD_2023_I, OBD_2024_I","2023.0, 2023.0, 2018.0","2024.0, 2024.0, 2026.0","f34786e7-0953-4085-9f3d-955481cdd4ea, 6d3a545c...","1, 1, 1","m3, kg, qm","09d32545-f3a5-4e53-b0a9-ac9fec1f24fb, eda3a44d...","Extrudiertes Polystyrol (XPS) m3, Polyurethan ...","500, 500.0","24.07, 6.2, 30.0","701.0, 1500, 1350.0",


from the above overview, we could drop the columns 'Schichtdicke (m)' (= only null) and 'Schuettdichte (kg/m3)' (= almost always null)



In [38]:
combined_TBS_OBD_3= combined_TBS_OBD_2.drop(columns=['Schichtdicke (m)','Schuettdichte (kg/m3)']) 
print(combined_TBS_OBD_2.shape) 
print(combined_TBS_OBD_3.shape)                           

(698, 29)
(698, 27)


### Reference to test the mapping method of TBS empty UUIDs = combined_TBS_OBD_3 dataframe

In [ ]:
# find TBS columns
TBS_columns=list(tbaustoff_df.columns)
tbaustoff_df.shape

(815, 11)

#### reference test = mapped dataframe between TBS with OBDs

In [ ]:
# keep TBS columns only
ref_mapped_TBS_df = combined_TBS_OBD_3[TBS_columns]
print(ref_mapped_TBS_df.shape)
ref_mapped_TBS_df.head(1)	

(698, 11)


,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor
0,3b1f0c75-07a4-4182-b310-5529fc5b54a6,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.0
1,0dec19c7-53db-4cb9-bbba-4d44d3da62a9,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.0


In [74]:
# Save reference table of mapped TBS
ref_mapped_TBS_df.to_csv("Reference_results_mapped_TBS_products_with_OBD_genericUUIDs.csv", index=False)

#### reference test = UNMAPPED dataframe between TBS with OBDs

In [ ]:
ref_unmapped_TBS_df= ref_mapped_TBS_df.copy()
print(ref_unmapped_TBS_df.duplicated().sum())
print(ref_unmapped_TBS_df.shape)
ref_unmapped_TBS_df.head(1)

0
(698, 11)


,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor
0,3b1f0c75-07a4-4182-b310-5529fc5b54a6,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.0
1,0dec19c7-53db-4cb9-bbba-4d44d3da62a9,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.0


In [85]:
columns_to_empty = ['oekobaudatProcessUuid','oekobaudatDatastockUuid','oekobaudatDatastockName']

for col in columns_to_empty:
    ref_unmapped_TBS_df[col] = ''

In [ ]:
print(ref_unmapped_TBS_df.shape)
print(ref_unmapped_TBS_df.duplicated().sum())
ref_unmapped_TBS_df.head(1)

(698, 11)
438


,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor
0,,,,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.0
1,,,,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.0


In [ ]:
ref_unmapped_TBS_df_2=ref_unmapped_TBS_df.drop_duplicates()
print(ref_unmapped_TBS_df_2.shape)
print(ref_unmapped_TBS_df_2.duplicated().sum())
ref_unmapped_TBS_df_2.head(1)

(260, 11)
0


,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor
0,,,,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.00
2,,,,2,2024-Q4,Aluminium Profil,2,Alu unbeschichtet od. eloxiert (Sz. Knetleg zu...,RC+,CL+,0.75


In [89]:
# Save reference table of unmapped TBS
ref_unmapped_TBS_df_2.to_csv("Reference_results_UNmapped_TBS_products_with_OBD_genericUUIDs.csv", index=False)